In [2]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import gym


In [3]:
env=gym.make('MsPacman-v0')
print(env.observation_space)

Box(0, 255, (210, 160, 3), uint8)


In [4]:
n_height = 210
n_width = 160
n_depth = 3
n_shape = [n_height,n_width,n_depth]
n_inputs = n_height * n_width * n_depth
env.frameskip = 4
# frameskip of 4 speeds up the process

In [5]:
from collections import deque 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# build the Q-Network
model = Sequential()
model.add(Flatten(input_shape = n_shape))
model.add(Dense(512, activation='relu',name='hidden1'))
model.add(Dense(9, activation='softmax', name='output'))
model.compile(loss='categorical_crossentropy',optimizer='adam')
model.summary()
q_nn =model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 100800)            0         
_________________________________________________________________
hidden1 (Dense)              (None, 512)               51610112  
_________________________________________________________________
output (Dense)               (None, 9)                 4617      
Total params: 51,614,729
Trainable params: 51,614,729
Non-trainable params: 0
_________________________________________________________________


In [7]:
def policy_q_nn(obs, env):
    
    if np.random.random() < explore_rate:
        action = env.action_space.sample()
    
    else:
        action = np.argmax(q_nn.predict(np.array([obs])))
    return action
# eps-greedy



In [23]:
def episode(env, policy, r_max=0, t_max=0):
    
    #restart
    obs = env.reset()
    state_prev = obs
    episode_reward = 0
    done = False
    t = 0
    env.step(0)
    
    #tf-fitting
    while not done:
        
        action = policy(state_prev, env)
        obs, reward, done, info = env.step(action)
        
        state_next = obs

        memory.append([state_prev,action,reward,state_next,done])

        states = np.array([x[0] for x in memory])
        states_next = np.array([np.zeros(n_shape) if x[4] else x[3] for x in memory])        
        q_values = q_nn.predict(states)
        q_values_next = q_nn.predict(states_next)
        
        for i in range(len(memory)):
            state_prev,action,reward,state_next,done = memory[i]
            if done:
                
                q_values[i,action] = reward
            else:
                
                best_q = np.amax(q_values_next[i])
                bellman_q = reward + discount_rate * best_q
                q_values[i,action] = bellman_q
                
        q_nn.fit(states,q_values,epochs=1,batch_size=50,verbose=0)
        state_prev = state_next
        

        episode_reward += reward
        if r_max > 0 and episode_reward > r_max:
            break
        t+=1
        if t_max > 0 and t == t_max:
            break
    return episode_reward




In [24]:

def experiment(env, policy, n_episodes,r_max=0, t_max=0):
    
    rewards=np.empty(shape=[n_episodes])
    for i in range(n_episodes):
        val = episode(env, policy, r_max, t_max)
        rewards[i]=val
            
    print('Policy:{}, Min reward:{}, Max reward:{}, Average reward:{}'
        .format(policy.__name__,
              np.min(rewards),
              np.max(rewards),
              np.mean(rewards)))

In [ ]:


discount_rate = 0.9
explore_rate = 0.2
n_episodes = 50


memory = deque(maxlen=1000)

experiment(env, policy_q_nn, n_episodes)

In [ ]:
env.close()